In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/12 22:28:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
home_sales_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView("homes_sales_df_temp")


In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

average_price=spark.sql('''
select year(date) as Year,
Round(avg(price),2) as AveragePriceFourBedrooms
from homes_sales_df_temp
where bedrooms = 4
group by
year(date)
order by
year(date)
''')
average_price.show()

+----+------------------------+
|Year|AveragePriceFourBedrooms|
+----+------------------------+
|2019|                300263.7|
|2020|               298353.78|
|2021|               301819.44|
|2022|               296363.88|
+----+------------------------+



In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
three_br_average_price=spark.sql('''
select year(date) as Year,
Round(avg(price),2) as AveragePrice3Bed3Bathroom
from homes_sales_df_temp
where bedrooms= 3 and bedrooms=3
group by year(date)
order by year(date)
''')
three_br_average_price.show()

+----+-------------------------+
|Year|AveragePrice3Bed3Bathroom|
+----+-------------------------+
|2019|                296327.09|
|2020|                302175.49|
|2021|                 300962.5|
|2022|                304872.51|
+----+-------------------------+



In [7]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
new_average_price=spark.sql('''
select year(date) as Year,
Round(avg(price),2) as AveragePrice
from homes_sales_df_temp
where bedrooms= 3 and bedrooms=3 and floors=2 and sqft_living>=2000
group by year(date)
order by year(date)
''')
new_average_price.show()

+----+------------+
|Year|AveragePrice|
+----+------------+
|2019|   302858.75|
|2020|   300936.22|
|2021|   304151.46|
|2022|   316003.25|
+----+------------+



In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

newer_average_price=spark.sql('''
select view as View,
Round(avg(price),2) as AveragePrice
from homes_sales_df_temp
where price >= 350000
group by view
order by view
''')
newer_average_price.show()
print("--- %s seconds ---" % (time.time() - start_time))

+----+------------+
|View|AveragePrice|
+----+------------+
|   0|   403848.51|
|   1|   401044.25|
|  10|   401868.43|
| 100|   1026669.5|
|  11|   399548.12|
|  12|   401501.32|
|  13|   398917.98|
|  14|   398570.03|
|  15|    404673.3|
|  16|   399586.53|
|  17|   398474.49|
|  18|   399332.91|
|  19|   398953.17|
|   2|   397389.25|
|  20|   399522.81|
|  21|   399758.88|
|  22|   402022.68|
|  23|   403411.92|
|  24|   400284.92|
|  25|   401298.69|
+----+------------+
only showing top 20 rows

--- 1.8100740909576416 seconds ---


In [10]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE homes_sales_df_temp")

DataFrame[]

In [11]:
# 8. Check if the table is cached.
spark.catalog.isCached('homes_sales_df_temp')

True

In [12]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

newer_average_price=spark.sql('''
select view as View,
Round(avg(price),2) as AveragePrice
from homes_sales_df_temp
where price >= 350000
group by view
order by view
''')
newer_average_price.show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+------------+
|View|AveragePrice|
+----+------------+
|   0|   403848.51|
|   1|   401044.25|
|  10|   401868.43|
| 100|   1026669.5|
|  11|   399548.12|
|  12|   401501.32|
|  13|   398917.98|
|  14|   398570.03|
|  15|    404673.3|
|  16|   399586.53|
|  17|   398474.49|
|  18|   399332.91|
|  19|   398953.17|
|   2|   397389.25|
|  20|   399522.81|
|  21|   399758.88|
|  22|   402022.68|
|  23|   403411.92|
|  24|   400284.92|
|  25|   401298.69|
+----+------------+
only showing top 20 rows

--- 0.6596672534942627 seconds ---


In [13]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
output_path='Home_Sale'

homes_sales_df_temp.write.partitionBy("date_built").parquet(output_path, mode="overwrite")

NameError: name 'homes_sales_df_temp' is not defined

In [ ]:
# 11. Read the formatted parquet data.
df = spark.read.parquet(homes_sales_df_temp)

In [ ]:
# 12. Create a temporary table for the parquet data.


In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()


newer_average_price=spark.sql('''
select view as View,
Round(avg(price),2) as AveragePrice
from homes_sales_df_temp
where price >= 350000
group by view
order by view
''')
newer_average_price.show()


print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 14. Uncache the home_sales temporary table.


In [ ]:
# 15. Check if the home_sales is no longer cached

